# Gather News via API

This notebook gathers the most recent week of US tech news via NewsAPI.
The news are stored in a PostgreSQL database.

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
import re
import datetime
from newsapi import NewsApiClient
from tqdm.notebook import tqdm_notebook
from configparser import ConfigParser
import sqlalchemy
from sqlalchemy import create_engine

## Extract from API

In [2]:
# Get keys from config file
parser = ConfigParser()
_ = parser.read('keys.cfg')

In [3]:
# Get API key from config file
newsapi = NewsApiClient(api_key=parser.get('newsapi','api_key1'))

In [4]:
# Get list of sources
sources = newsapi.get_sources(language='en',country='us',category='technology')

In [5]:
# Unnest json
sources_df = pd.json_normalize(sources,record_path=['sources'])

In [ ]:
sources_df.head()

In [7]:
# API call parameters
from_date = (datetime.date.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
to_date = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
pages = range(1,5)

In [8]:
# Get 1 page of news
def get_news(p):
    news_dict = {}
    sources = sources_df['id']
    for s in sources:
        response = newsapi.get_everything(sources=s,
                                          page=p,
                                          from_param=from_date,
                                          to=to_date,
                                          language='en',
                                          sort_by='popularity')
        flatten = pd.json_normalize(response,record_path=['articles'])
        news_dict[s] = flatten
    news_df = pd.concat([news_dict[s] for s in sources],ignore_index=True)        
    return news_df

In [9]:
# Get multiple pages of news
def get_news_multiple_pages(pages):
    news_dict = {}
    for p in tqdm_notebook(pages):
        news_dict[p] = get_news(p)
    news_df = pd.concat([news_dict[p] for p in pages],ignore_index=True)
    return news_df

In [ ]:

# Call get news function
news_df = get_news_multiple_pages(pages)

In [ ]:
news_df.head()

## Save in PostgreSQL database

In [12]:
# Connect to database
engine = create_engine(parser.get('news','conn_string'))

In [ ]:
# Append new entries to news, drop duplicates
existing_df = pd.read_sql_query('SELECT * FROM news', engine)
merged_df = pd.concat([existing_df, news_df]).drop_duplicates()
merged_df.to_sql('news', con=engine, if_exists='replace', index=False)

In [ ]:
# Append new entries to sources, drop duplicates
existing_df = pd.read_sql_query('SELECT * FROM sources', engine)
merged_df = pd.concat([existing_df, sources_df]).drop_duplicates()
merged_df.to_sql('sources', con=engine, if_exists='replace', index=False)

## Read from database

In [19]:
# Read from the database and turn into dataframe
news_df = pd.read_sql_query('select * from "news"',con=engine)
sources_df = pd.read_sql_query('select * from "sources"',con=engine)

In [ ]:
news_df.head()

In [ ]:
sources_df.head()